## Face Recognition Model prior to Data Cleaning and XAI Implementation

In [34]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from FaceRecognitionModel import FaceRecognitionRNN

### Load Dataset

In [39]:
# Custom dataset class
class LFWDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

lfw_path = r'lfw_data\lfw'
input_size = 4096  # Adjusted for 64x64 grayscale image flattened

# Loading function
def load_lfw_data():
    X, y = [], []
    for person_dir in os.listdir(lfw_path):
        person_path = os.path.join(lfw_path, person_dir)
        for image_file in os.listdir(person_path):
            if image_file.endswith('.jpg'):
                image_path = os.path.join(person_path, image_file)
                image = Image.open(image_path).convert('L').resize((64, 64))
                X.append(np.array(image).flatten())
                y.append(person_dir)
    return np.array(X), np.array(y)

# Load the data
X, y = load_lfw_data()


In [40]:
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

### Split the dataset

In [41]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create PyTorch datasets and dataloaders
train_dataset = LFWDataset(X_train, y_train)
test_dataset = LFWDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Training the Model

In [49]:
# Training the model
hidden_size = 256
num_layers = 1
num_classes = len(np.unique(y))
num_epochs = 10
learning_rate = 0.001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FaceRecognitionRNN(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 9.1564
Epoch [2/10], Loss: 8.2221
Epoch [3/10], Loss: 8.1416
Epoch [4/10], Loss: 7.2962
Epoch [5/10], Loss: 7.6558
Epoch [6/10], Loss: 8.4862
Epoch [7/10], Loss: 7.8811
Epoch [8/10], Loss: 7.9729
Epoch [9/10], Loss: 8.4762
Epoch [10/10], Loss: 8.1226


### Evaluating the Model

In [51]:
from sklearn.metrics import accuracy_score

# Set the model to evaluation mode
model.eval()

# Lists to store true labels and predictions
all_labels = []
all_predictions = []

with torch.no_grad():  # Disable gradient calculation for evaluation
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

        # Get the predicted class by taking the index of the max log-probability
        _, predicted = torch.max(outputs.data, 1)

        # Append the true labels and predictions to the lists
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate the accuracy
accuracy = accuracy_score(all_labels, all_predictions)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 4.27%
